## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_challenge import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
import os

In [8]:
city_data_to_load = os.path.join("c:", os.sep, "Users", "Juijie", "Desktop", "Analysis Projects", "M-6", "Weather_Database", "WeatherPy_database.csv")
print(city_data_to_load)

c:\Users\Juijie\Desktop\Analysis Projects\M-6\Weather_Database\WeatherPy_database.csv


In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,2021-12-23 17:20:45,42.6001,-73.9662,29.64,36,55,5.01,broken clouds
1,1,Placerville,US,2021-12-23 17:35:06,38.7296,-120.7986,51.67,97,90,1.01,moderate rain
2,2,Kaitangata,NZ,2021-12-23 17:35:06,-46.2817,169.8464,51.93,85,95,2.89,overcast clouds
3,3,Dayong,CN,2021-12-23 17:35:06,25.0209,118.2898,62.89,73,100,2.39,overcast clouds
4,4,Ushuaia,AR,2021-12-23 17:30:09,-54.8000,-68.3000,60.46,41,75,3.44,broken clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for you trip?"))

What is the minimum temperature you would like for your trip?22
What is the maximum temperature you would like for you trip?38


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,2021-12-23 17:20:45,42.6001,-73.9662,29.64,36,55,5.01,broken clouds
7,7,Severo-Kurilsk,RU,2021-12-23 17:34:34,50.6789,156.1250,29.19,68,85,19.84,light snow
12,12,Jamestown,US,2021-12-23 17:24:46,42.0970,-79.2353,30.72,70,100,7.00,overcast clouds
15,15,Saint-Augustin,CA,2021-12-23 17:30:26,51.2260,-58.6502,27.19,93,100,23.91,snow
44,44,Sitka,US,2021-12-23 17:33:28,57.0531,-135.3300,28.29,86,90,0.00,mist


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                83
City                   83
Country                83
Date                   83
Lat                    83
Lng                    83
Max Temp               83
Humidity               83
Cloudiness             83
Wind Speed             83
Current Description    83
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,29.64,broken clouds,42.6001,-73.9662,
7,Severo-Kurilsk,RU,29.19,light snow,50.6789,156.1250,
12,Jamestown,US,30.72,overcast clouds,42.0970,-79.2353,
15,Saint-Augustin,CA,27.19,snow,51.2260,-58.6502,
44,Sitka,US,28.29,mist,57.0531,-135.3300,
45,Kemin,KG,34.32,light snow,42.7846,75.6892,
55,Xining,CN,27.18,overcast clouds,36.6167,101.7667,
58,Qaanaaq,GL,25.95,overcast clouds,77.4840,-69.3632,
81,Brainerd,US,26.55,clear sky,46.3580,-94.2008,
82,Nanortalik,GL,30.74,overcast clouds,60.1432,-45.2371,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")
        

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [21]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,29.64,broken clouds,42.6001,-73.9662,
7,Severo-Kurilsk,RU,29.19,light snow,50.6789,156.1250,Kak Doma
12,Jamestown,US,30.72,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
15,Saint-Augustin,CA,27.19,snow,51.2260,-58.6502,
44,Sitka,US,28.29,mist,57.0531,-135.3300,Westmark Sitka
...,...,...,...,...,...,...,...
645,Kenai,US,33.85,clear sky,60.5544,-151.2583,Aspen Extended Stay Suites
648,Nor Geghi,AM,24.46,overcast clouds,40.3210,44.5803,ARAME
655,Marshfield,US,31.44,few clouds,44.6689,-90.1718,"Holiday Inn Conference Ctr Marshfield, an IHG ..."
656,Paamiut,GL,31.46,overcast clouds,61.9940,-49.6678,Pistut


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))